In [3]:
import netCDF4 as nc
import sys
import glob
import csv
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)


整理下里面的时间点信息，特别容易混乱

    （1）这里每次说的给的数据集都会 从3：00（北京时11:00）开始（从北京11点到北京第二天晚上11点就是37个小时），这是因为每次数值预报都是产生预测结果
    （2）每次数值预测时预测到第二天15:00（北京时23:00），也就是预测了37个小时。
    （3）在提交时候，会在当天的12:00（北京时20:00）开放数据集，包含从8月29日至上述日期当天的“观测”和“睿图”数据集。 于当天18:00（北京时第二天02:00）前提交至平台。 预测到第二天15:00（北京时23:00）的天气情况。
    （4）看看那个解释图，其中有三个时间点3：00（北京时11:00） 、6：00（北京时14:00）、第二天15:00（北京时23:00）
     
    疑问：
    *很明显在晚上20点给你发放数据集，那么睿图数据（天气预报）是已经给出过的，这36个小时包含了到预测结果的第二天23点的所有预报信息。。。。但是哎，有点不明白，实时的观测数据应该是没有未来那段时间的信息的？？？？？
    明白了，实时数据给了要预测的37个小时中的前三个小时的数据信息，后面的34个小时信息则是自己要预测的。但是在提交时候是需要37个小时一起提交（也就是说有3个小时数据是已知的，拼接进去的）
    *我们可以发现特别说明里面的那个图，看样子要预测的是14点到第二天23的数据情况，但是呢这里在提交时候要提交第一天11点到第二天23点的情况。  那么意思是要预测36个小时的情况？？？？？？
    
    补充：因为一次是给出36个小时的情况，所以相邻两天的36个小时是有重叠时间的，其中观测数据里面重叠的时间内数据是相同的，而睿图数据中重叠的时间是不同的，因为是不同时刻下的预测计算的结果。

In [4]:
#所有提供的数据集， 基本的两个训练和验证、    两个双周赛的数据
the_test_data=nc.Dataset('../data/ai_challenger_wf2018_testa1_20180829-20180924.nc')
the_train_data=nc.Dataset('../data/ai_challenger_wf2018_trainingset_20150301-20180531.nc')
the_val_data=nc.Dataset('../data/ai_challenger_wf2018_validation_20180601-20180828_20180905.nc')
the_testB_data=nc.Dataset('../data/ai_challenger_weather_testingsetB_20180829-20181015.nc')
print('展示测试集周1:',the_test_data)
print('展示训练集:',the_train_data)
print('展示验证集:',the_val_data)
print('展示测试集周2:',the_testB_data)
#print('单个字段的展示：',the_test_data['d10m_M'])
print('展示测试集其中的variables:',the_test_data.variables)
print('展示训练集其中的variables:',the_train_data.variables)
print('展示验证集其中的variables:',the_val_data.variables)
print('展示测试集B其中的variables:',the_testB_data.variables)
#发现其中包含可很多种字段  float32 d10m_M(date,foretimes,station) ......  字段中括号里第一个表示数据，第二个表示日期，第三个表示站点

展示测试集周1: <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): foretimes(37), station(10), date(27)
    variables(dimensions): int32 foretimes(foretimes), int32 station(station), float64 date(date), float32 psfc_M(date,foretimes,station), float32 t2m_M(date,foretimes,station), float32 q2m_M(date,foretimes,station), float32 rh2m_M(date,foretimes,station), float32 w10m_M(date,foretimes,station), float32 d10m_M(date,foretimes,station), float32 u10m_M(date,foretimes,station), float32 v10m_M(date,foretimes,station), float32 SWD_M(date,foretimes,station), float32 GLW_M(date,foretimes,station), float32 HFX_M(date,foretimes,station), float32 LH_M(date,foretimes,station), float32 RAIN_M(date,foretimes,station), float32 PBLH_M(date,foretimes,station), float32 TC975_M(date,foretimes,station), float32 TC925_M(date,foretimes,station), float32 TC850_M(date,foretimes,station), float32 TC700_M(date,foretimes,station), float32 TC500_M(da

# 字段介绍
    “观测”集逐时记录当前气象观测站点的9个地面气象要素，通过气象仪器实时监测得到；“睿图”集包含地面和特征气压层共计29个气象要素，由数值预报模式在超级计算机上运算产生，其在每天03:00（北京时11:00）启动区域数值模式，预报至第二天15:00（北京时23:00），共计37个时次（00–36）。
    要预测的目标是:
                t2m	        地面以上2米高度处温度(摄氏度℃)	[-40.0, 55.0]
                rh2m	    地面以上2米高度处相对湿度(百分比%)	[0.0,100.0]
                w10m	    地面以上10米高度处风速(米每秒m/s)	[0.0, 30.0]
    其中9个观察集数据为：
                psur_obs	地面气压(百帕hPa)	[850.0, 1100.0]
                t2m_obs	    地面以上2米高度处温度(摄氏度℃)	[-40.0, 55.0]
                q2m_obs	    地面以上2米高度处比湿(克每千克g/kg)	[0.0, 30.0]
                rh2m_obs	地面以上2米高度处相对湿度(百分比%)	[0.0,100.0]
                w10m_obs	地面以上10米高度处风速(米每秒m/s)	[0.0, 30.0]
                d10m_obs	地面以上10米高度处风向(角度°)	[0.0, 360.0]
                u10m_obs	地面以上10米高度处经向风(米每秒m/s)	[-30.0, 30.0]
                v10m_obs	地面以上10米高度处纬向风(米每秒m/s)	[-30.0, 30.0]
                RAIN_obs	地面1小时累计降水量(毫米mm)	[0.0, 400.0]
    29个睿图详细数据分别为：
                psfc_M	   地面气压(百帕hPa)	[850.0, 1100.0]
                t2m_M	   地面以上2米高度处温度(摄氏度℃)	[-40.0, 55.0]
                q2m_M	   地面以上2米高度处比湿(克每千克g/kg)	[0.0, 30.0]
                rh2m_M	   地面以上2米高度处相对湿度(百分比%)	[0.0,100.0]
                w10m_M	   地面以上10米高度处风速(米每秒m/s)	[0.0, 30.0]
                d10m_M	   地面以上10米高度处风向(角度°)	[0.0, 360.0]
                u10m_M	   地面以上10米高度处经向风(米每秒m/s)	[-30.0, 30.0]
                v10m_M	   地面以上10米高度处纬向风(米每秒m/s)	[-30.0, 30.0]
                SWD_M	   地面处下行短波辐射量(瓦特每平方米W/m2)	[0.0, 1500.0]
                GLW_M	   地面处下行长波辐射量(瓦特每平方米W/m2)	[0.0, 800.0]
                HFX_M	   地面感热扰动(瓦特每平方米W/m2) [	[-400.0, 1000.0]
                LH_M	   地面潜热扰动(瓦特每平方米W/m2)	[-100.0, 1000.0]
                RAIN_M	   地面1小时累计降水量(毫米mm)	[0.0, 400.0]
                PBLH_M	   边界层高度(米m)	[0.0, 6000.0]
                TC975_M	   气压层975百帕的温度(摄氏度℃)	[-50.0, 45.0]
                TC925_M	   气压层925百帕的温度(摄氏度℃)	[-50.0, 45.0]
                TC850_M	   气压层850百帕的温度(摄氏度℃)	[-55.0, 40.0]
                TC700_M	   气压层700百帕的温度(摄氏度℃)	[-60.0, 35.0]
                TC500_M	   气压层500百帕的温度(摄氏度℃)	[-70.0, 30.0]
                wspd975_M  气压层975百帕的风速(米每秒m/s)	[0.0, 60.0]
                wspd925_M  气压层925百帕的风速(米每秒m/s)	[0.0, 70.0]
                wspd850_M  气压层850百帕的风速(米每秒m/s)	[0.0, 80.0]
                wspd700_M  气压层700百帕的风速(米每秒m/s)	[0.0, 90.0]
                wspd500_M  气压层500百帕的风速(米每秒m/s)	[0.0, 100.0]
                Q975_M	   气压层975百帕的比湿(克每千克g/kg)	[0.0, 30.0]
                Q925_M	   气压层925百帕的比湿(克每千克g/kg)	[0.0, 30.0]
                Q850_M	   气压层850百帕的比湿(克每千克g/kg)	[0.0, 30.0]
                Q700_M	   气压层700百帕的比湿(克每千克g/kg)	[0.0, 25.0]
                Q500_M	   气压层500百帕的比湿(克每千克g/kg)	[0.0, 25.0]
      详细下来字段表示为： 
                int32 foretimes(foretimes),               时间点编号 从0-36共37个
                int32 station(station),                   10个站点编号
                float64 date(date),                       哪一天预测的，对应于所有数据的第一个编号，
                float32 psfc_M(date,foretimes,station), 
                float32 t2m_M(date,foretimes,station), 
                float32 q2m_M(date,foretimes,station), 
                float32 rh2m_M(date,foretimes,station), 
                float32 w10m_M(date,foretimes,station), 
                float32 d10m_M(date,foretimes,station),
                float32 u10m_M(date,foretimes,station), 
                float32 v10m_M(date,foretimes,station), 
                float32 SWD_M(date,foretimes,station), 
                float32 GLW_M(date,foretimes,station), 
                float32 HFX_M(date,foretimes,station),
                float32 LH_M(date,foretimes,station), 
                float32 RAIN_M(date,foretimes,station), 
                float32 PBLH_M(date,foretimes,station), 
                float32 TC975_M(date,foretimes,station),
                float32 TC925_M(date,foretimes,station), 
                float32 TC850_M(date,foretimes,station), 
                float32 TC700_M(date,foretimes,station), 
                float32 TC500_M(date,foretimes,station), 
                float32 wspd975_M(date,foretimes,station), 
                float32 wspd925_M(date,foretimes,station), 
                float32 wspd850_M(date,foretimes,station),
                float32 wspd700_M(date,foretimes,station), 
                float32 wspd500_M(date,foretimes,station),
                float32 Q975_M(date,foretimes,station), 
                float32 Q925_M(date,foretimes,station),
                float32 Q850_M(date,foretimes,station),
                float32 Q700_M(date,foretimes,station), 
                float32 Q500_M(date,foretimes,station),
                float32 psur_obs(date,foretimes,station), 
                float32 t2m_obs(date,foretimes,station),
                float32 q2m_obs(date,foretimes,station), 
                float32 w10m_obs(date,foretimes,station), 
                float32 d10m_obs(date,foretimes,station), 
                float32 rh2m_obs(date,foretimes,station),
                float32 u10m_obs(date,foretimes,station),
                float32 v10m_obs(date,foretimes,station),
                float32 RAIN_obs(date,foretimes,station)
int32 foretimes(foretimes), int32 station(station), float64 date(date)

# 对测试集进行观察：


    *使用the_test_data.variables可以得到整个nc文件中的情况，得到每个字段的描述情况，包含数据类型和数据维度，是个存储时序信息的文件。

    *通过进行观察可以发现很多字段的形式类似 ： float32 TC850_M(date,foretimes,station)，  其中总体的nc文件中包含了很多的字段，这些字段中大部分形式为
    （1）date：数据日期（国际时UTC）

    （2）foretimes：数据时次（00–36，默认为37个时次，时间间隔为1小时）

    （3）stations：站点编号

    从中我们可以发现，对于有些字段其数据shape是(27, 37, 10)，表示的是27天， 每个天都是37小时，10表示会有10个站点需要进行预测。
    这里每天整37个小时的原因是：

          “观测”集逐时记录当前气象观测站点的9个地面气象要素，通过气象仪器实时监测得到；“睿图”集包含地面和特征气压层共计29个气象要素，由数值预报模式在超级计算机上运算产生，*其在每天03:00（北京时中午11:00）启动区域数值模式，预报至第二天15:00（北京时23:00），共计37个时次（00–36）*。两者区别为：前者仅记录当前气象要素实况；后者可预测未来36小时内气象要素估计值，但存在误差。
          
# 对训练集进行观察
     *训练集基本形式跟测试集是 相同的，包含的字段基本是一样的，但是这里面每个字段的维度大多是(1188, 37, 10)，这是因为训练数据比较充足，总共包含了3年多个气象数据，数据字段基本和测试集是相同的。
     别人算法工程师在装进这个包时很多字段已经说得很清楚了，如下是一些描述：
     举例观察其中一个字段，float32 d10m_M(date, foretimes, station)
        units: degree
        FillValue: -9999.0
        description: Direction of wind at 10M
        unlimited dimensions: 
        current shape = (27, 37, 10)
        filling off
    此外对于变量和其属性还有以下的说明：
    psfc_M(date, foretimes, station)。上述三个维度值在Dimensions部分给出。
    psfc_M的属性包括单位名称（units）、缺省填充值（_FillValue）、变量的描述信息（description）。它们是对该物理量的补充说明。
# 目标 
      注意要区分好因为这是全球赛，所以赛题上说的15点对应的是北京时间的23点， 我们要利用天气预报数据（睿图计算结果）计算出第一天6点到第二天15点的数据，可以详细看赛题上的图其中要预测部分的数据睿图都已经给出了，但是观测集合上只给了前面一小部分作为预测的依据。
      
      注意下面这句话，说的很清楚，训练集合验证集里面都是历史数据，而测试集A和测试集B是用于双周赛时候预测使用的。
      
      相比训练集和验证集，测试集在“观测”数据集的组成上有所区别。训练集和验证集数据均以历史气象数据为基础，用于气象预报模型的建立，因此在两者发布时，“观测”集可以获得与“睿图”集的数据时间相对应的气象要素观测值。然而，测试集A和B集用于双周赛和决赛，用于实时天气预报，因此每个测试集的最后一天的“观测”集，总会出现特别说明1中的情况，即图中红色时间段内观测值被设为缺省值（-9999.f），这段时间也即本次竞赛的气象要素预报时段。

In [19]:
# 观察d10m_M字段情况      同样是27 37  10，但是明白了，这里观测值也都有数据是因为这是历史的都已经知道了，但是这里
the_d10m_M=np.asarray(the_test_data['d10m_M'])
print('睿图数据字段维度',the_d10m_M.shape)#可以发现就是这个字段，那么
print('睿图数据字段维度',np.asarray(the_val_data['d10m_M']).shape)
print('睿图数据字段维度',np.asarray(the_train_data['d10m_M']).shape)
print('睿图数据字段维度',np.asarray(the_testB_data['d10m_M']).shape)
psur_obs=np.asarray(the_test_data['psur_obs'])
print('单周A提供的字段psur_obs数据：',np.array(the_testB_data['station']))  #

print('测试单周A最后一天提供的第二个小时的数据',psur_obs[26][2]) 
print('测试单周A最后一天提供的第三个小时的数据',psur_obs[26][3])
print('测试单周A最后一天提供的第三个小时的数据',psur_obs[26][4])
print('测试单周A最后一天提供的第八个小时的数据',psur_obs[26][8]) 
#the_d10m_M[0][0]#第一个小时的10个站点的地面以上10米高度处风向(角度°)    发现还是有不少异常值的


np.asarray(the_test_data['date'])


睿图数据字段维度 (27, 37, 10)
睿图数据字段维度 (89, 37, 10)
睿图数据字段维度 (1188, 37, 10)
睿图数据字段维度 (48, 37, 10)
单周A提供的字段psur_obs数据： [90001 90002 90003 90004 90005 90006 90007 90008 90009 90010]
测试单周A最后一天提供的第二个小时的数据 [1017.5 1018.   986.7 1012.7 1012.1  973.3  981.9  883.5 1017.7  969. ]
测试单周A最后一天提供的第三个小时的数据 [1016.6 1017.1  985.8 1011.7 1011.3  972.5  981.   883.  1016.7  968.2]
测试单周A最后一天提供的第三个小时的数据 [-9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999.]
测试单周A最后一天提供的第八个小时的数据 [-9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999. -9999.]


array([2.0180829e+09, 2.0180830e+09, 2.0180831e+09, 2.0180901e+09,
       2.0180902e+09, 2.0180903e+09, 2.0180904e+09, 2.0180905e+09,
       2.0180906e+09, 2.0180907e+09, 2.0180908e+09, 2.0180909e+09,
       2.0180910e+09, 2.0180911e+09, 2.0180912e+09, 2.0180913e+09,
       2.0180914e+09, 2.0180915e+09, 2.0180916e+09, 2.0180917e+09,
       2.0180918e+09, 2.0180919e+09, 2.0180920e+09, 2.0180921e+09,
       2.0180922e+09, 2.0180923e+09, 2.0180924e+09])

In [54]:
#观察四个数据集得时间点信息 
the_test_data_date=np.asarray(the_test_data['date'])
the_train_data_date=np.asarray(the_train_data['date'])
the_val_data_date=np.asarray(the_val_data['date'])
the_testB_data_date=np.asarray(the_testB_data['date'])
print('展示测试集周1:',[str(i) for i in the_test_data_date])
print('展示训练集:',[str(i) for i in the_train_data_date])
print('展示验证集:',[str(i) for i in the_val_data_date])
print('展示测试集周2:',[str(i) for i in the_testB_data_date])
print('对于双周赛  展示睿图对最后一小时的预测:',the_testB_data['d10m_M'][47][36])
print('对于双周赛  展示精准观测对前后前40小时的预测:',the_testB_data['rh2m_obs'][46][4])
print('对于双周赛  展示这个观测点开始:',the_testB_data['rh2m_obs'][47][0])
print('对于双周赛  展示精准观测对前后前30小时的预测:',the_testB_data['rh2m_obs'][47][6])
print('对于双周赛  展示精准观测对前后前20小时的预测:',the_testB_data['rh2m_obs'][47][16])
print('对于双周赛  展示精准观测对最后一小时的预测:',the_testB_data['rh2m_obs'][47][36])


展示测试集周1: ['2018082903.0', '2018083003.0', '2018083103.0', '2018090103.0', '2018090203.0', '2018090303.0', '2018090403.0', '2018090503.0', '2018090603.0', '2018090703.0', '2018090803.0', '2018090903.0', '2018091003.0', '2018091103.0', '2018091203.0', '2018091303.0', '2018091403.0', '2018091503.0', '2018091603.0', '2018091703.0', '2018091803.0', '2018091903.0', '2018092003.0', '2018092103.0', '2018092203.0', '2018092303.0', '2018092403.0']
展示训练集: ['2015030103.0', '2015030203.0', '2015030303.0', '2015030403.0', '2015030503.0', '2015030603.0', '2015030703.0', '2015030803.0', '2015030903.0', '2015031003.0', '2015031103.0', '2015031203.0', '2015031303.0', '2015031403.0', '2015031503.0', '2015031603.0', '2015031703.0', '2015031803.0', '2015031903.0', '2015032003.0', '2015032103.0', '2015032203.0', '2015032303.0', '2015032403.0', '2015032503.0', '2015032603.0', '2015032703.0', '2015032803.0', '2015032903.0', '2015033003.0', '2015033103.0', '2015040103.0', '2015040203.0', '2015040303.0', '20150

观察时间点发现，无论训练集、验证集、测试集都是从国际时间3点（每天03:00即北京时11:00）当做数值预测的起点，也就是睿图是每次在这个时候开始预测的，所以明白了吧，就是这里我们是根据国际时间3点之前的数据开始预测，预测从国际3点到次日15点的天气预报情况。

可以明确的发现，已经很明白了，未来那些小时的

In [20]:
np.array(the_train_data['psfc_M'])

array([[[-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ],
        [-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ],
        [-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ],
        ...,
        [-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ],
        [-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ],
        [-9999.   , -9999.   , -9999.   , ..., -9999.   , -9999.   ,
         -9999.   ]],

       [[ 1008.616,  1011.787,   980.865, ...,   927.181,  1007.82 ,
           948.221],
        [ 1005.951,  1008.579,   976.324, ...,   924.636,  1005.99 ,
           946.07 ],
        [ 1005.395,  1008.209,   976.387, ...,   924.19 ,  1005.875,
           945.99 ],
        ...,
        [ 1021.736,  1025.081,   991.159, ...,   936.496,  1022.929,
           960.625],
        [ 1022.347,  1025.9  ,   991.369, ...,   937.078,  1023.464,
   